In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_recall_curve, accuracy_score
import cv2
import numpy as np
import os

# Pixelation function
def pixelate_image(image, scale=0.1):
    height, width, _ = image.shape
    small = cv2.resize(image, (int(width * scale), int(height * scale)), interpolation=cv2.INTER_LINEAR)
    return cv2.resize(small, (width, height), interpolation=cv2.INTER_NEAREST)

# Data Preparation
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    brightness_range=[0.8, 1.2],
    preprocessing_function=lambda x: pixelate_image(x, scale=0.1) if np.random.rand() > 0.5 else x
)

train_generator = datagen.flow_from_directory(
    '/kaggle/input/pixelated-image-detection-and-correction/Image_Processing',
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    '/kaggle/input/pixelated-image-detection-and-correction/Image_Processing',
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Compute class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = {i : class_weights[i] for i in range(len(class_weights))}

# Model Design
weights_path = '/kaggle/input/models/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_128_no_top.h5'
base_model = MobileNetV2(input_shape=(128, 128, 3), include_top=False, weights=None)

if os.path.exists(weights_path):
    base_model.load_weights(weights_path)
else:
    raise FileNotFoundError(f"Weights file not found at {weights_path}")

for layer in base_model.layers[:-20]:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)
]

# Model Training
try:
    history = model.fit(
        train_generator,
        epochs=30,
        validation_data=validation_generator,
        steps_per_epoch=train_generator.samples // 32,
        validation_steps=validation_generator.samples // 32,
        class_weight=class_weights,
        callbacks=callbacks
    )
except KeyboardInterrupt:
    print("Training interrupted. Saving model...")
    model.save('pixelation_detector_interrupted.h5')
    raise

# Evaluation
val_preds = model.predict(validation_generator)
val_labels = validation_generator.classes
val_preds_binary = (val_preds > 0.5).astype(int).flatten()

# Print classification report and confusion matrix
print(classification_report(val_labels, val_preds_binary))
print(confusion_matrix(val_labels, val_preds_binary))

# Calculate F1 score
f1 = f1_score(val_labels, val_preds_binary)
print(f"F1 Score: {f1:.2f}")

# Precision-Recall Curve
precision, recall, thresholds = precision_recall_curve(val_labels, val_preds)
print("Precision-Recall curve calculated.")

# Calculate accuracy
accuracy = accuracy_score(val_labels, val_preds_binary) * 100
print(f"Accuracy: {accuracy:.2f}%")

# Save and check model size
model_path = 'pixelation_detector.h5'
model.save(model_path)
model_size = os.path.getsize(model_path) / (1024 * 1024)  # Convert to MB
print(f"Model size: {model_size:.2f} MB")

# Usage Example
def load_and_predict(image_path, model_path='pixelation_detector.h5'):
    model = load_model(model_path)
    image = cv2.imread(image_path)
    image = cv2.resize(image, (128, 128))
    image = image / 255.0  # Rescale
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    prediction = model.predict(image)
    is_pixelated = prediction[0][0] > 0.5
    return is_pixelated

# Example usage
image_path = '/kaggle/input/pixelated-image-detection-and-correction/Image_Processing/Original/120.png'
is_pixelated = load_and_predict(image_path)
print(f"The image is {'pixelated' if is_pixelated else 'not pixelated'}")


Found 1992 images belonging to 2 classes.
Found 498 images belonging to 2 classes.
Epoch 1/30


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


62/62 ━━━━━━━━━━━━━━━━━━━━ 80s 1s/step - accuracy: 0.5243 - loss: 0.7895 - val_accuracy: 0.5437 - val_loss: 0.7211 - learning_rate: 1.0000e-04
Epoch 2/30
 1/62 ━━━━━━━━━━━━━━━━━━━━ 18s 302ms/step - accuracy: 0.6250 - loss: 0.6062

/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6250 - loss: 0.6062 - val_accuracy: 0.6667 - val_loss: 0.5967 - learning_rate: 1.0000e-04
Epoch 3/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 67s 996ms/step - accuracy: 0.6261 - loss: 0.6565 - val_accuracy: 0.5500 - val_loss: 0.7442 - learning_rate: 1.0000e-04
Epoch 4/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6562 - loss: 0.5091 - val_accuracy: 0.6111 - val_loss: 0.7182 - learning_rate: 1.0000e-04
Epoch 5/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - accuracy: 0.6298 - loss: 0.6156 - val_accuracy: 0.5667 - val_loss: 0.8492 - learning_rate: 1.0000e-04
Epoch 6/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6250 - loss: 0.7091 - val_accuracy: 0.6111 - val_loss: 0.7345 - learning_rate: 2.0000e-05
Epoch 7/30
62/62 ━━━━━━━━━━━━━━━━━━━━ 66s 1s/step - accuracy: 0.6316 - loss: 0.6485 - val_accuracy: 0.5396 - val_loss: 0.8589 - learning_rate: 2.0000e-05
16/16 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step
              precision    recall  f1-score  

In [9]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define constants
IMG_HEIGHT, IMG_WIDTH = 128, 128  # Input image dimensions after downsampling
BATCH_SIZE = 32
EPOCHS = 1

# Define paths to your dataset folders
train_dir = '/kaggle/input/pixelated-image-detection-and-correction/Image_Processing/'
train_pixelated_dir = train_dir + 'Pixelated'
train_original_dir = train_dir + 'Original'

# ImageDataGenerator for augmentation and scaling
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Splitting the data for validation
)

# Generate batches of augmented data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'  # Specify subset for training
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'  # Specify subset for validation
)

# Define the model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    
    GlobalAveragePooling2D(),
    
    Dense(64, activation='relu'),
    Dropout(0.5),
    
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(),
              loss=BinaryCrossentropy(),
              metrics=[BinaryAccuracy(), Precision(), Recall()])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE
)

# Save the model
model.save('/kaggle/working/pixelated_detection_model.h5')

# Calculate and print model size
import os
model_size = os.path.getsize('/kaggle/working/pixelated_detection_model.h5') / (1024 * 1024)  # in MB
print(f'Model Size: {model_size:.2f} MB')

# Evaluate the model
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/kaggle/input/pixelated-image-detection-and-correction/Image_Processing',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False  # Ensure to evaluate in order
)

evaluation_results = model.evaluate(test_generator)
loss, accuracy, precision, recall = evaluation_results[:4]
print(f'Test Accuracy: {accuracy * 100:.2f}%')

# Example of using the trained model for prediction
sample_image_path = '/kaggle/input/trials/trial/original.jpg'
sample_image = tf.keras.preprocessing.image.load_img(sample_image_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
sample_image = tf.keras.preprocessing.image.img_to_array(sample_image)
sample_image = np.expand_dims(sample_image, axis=0) / 255.0  # Rescale and expand dimensions
prediction = model.predict(sample_image)
if prediction[0] > 0.5:
    print('Prediction: Pixelated')
else:
    print('Prediction: Not Pixelated')


Found 1992 images belonging to 2 classes.
Found 498 images belonging to 2 classes.
62/62 ━━━━━━━━━━━━━━━━━━━━ 87s 1s/step - binary_accuracy: 0.5381 - loss: 0.7787 - precision_8: 0.5493 - recall_8: 0.4367 - val_binary_accuracy: 0.5146 - val_loss: 0.6923 - val_precision_8: 0.5676 - val_recall_8: 0.0882
Model Size: 1.24 MB
Found 2490 images belonging to 2 classes.
78/78 ━━━━━━━━━━━━━━━━━━━━ 55s 686ms/step - binary_accuracy: 0.7916 - loss: 0.6328 - precision_8: 0.1582 - recall_8: 0.0369
Test Accuracy: 50.20%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
Prediction: Not Pixelated


In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Define constants
IMG_HEIGHT, IMG_WIDTH = 128, 128  # Input image dimensions after downsampling
BATCH_SIZE = 32
EPOCHS = 50

# Define paths to your dataset folders
train_dir = '/kaggle/input/pixelated-image-detection-and-correction/Image_Processing/'
train_pixelated_dir = train_dir + 'Pixelated'
train_original_dir = train_dir + 'Original'

# ImageDataGenerator for augmentation and scaling
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Splitting the data for validation
)

# Generate batches of augmented data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'  # Specify subset for training
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'  # Specify subset for validation
)

# Define the model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D((2, 2)),
    BatchNormalization(),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),

    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),

    GlobalAveragePooling2D(),

    Dense(128, activation='relu'),
    Dropout(0.5),

    Dense(64, activation='relu'),
    Dropout(0.5),

    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss=BinaryCrossentropy(),
              metrics=[BinaryAccuracy(), Precision(), Recall()])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)
model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    callbacks=[early_stopping, reduce_lr, model_checkpoint]
)

# Save the model
model.save('/kaggle/working/pixelated_detection_model.keras')

# Calculate and print model size
import os
model_size = os.path.getsize('/kaggle/working/pixelated_detection_model.keras') / (1024 * 1024)  # in MB
print(f'Model Size: {model_size:.2f} MB')

# Evaluate the model
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/kaggle/input/pixelated-image-detection-and-correction/Image_Processing',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False  # Ensure to evaluate in order
)

evaluation_results = model.evaluate(test_generator)
loss, accuracy, precision, recall = evaluation_results[:4]
print(f'Test Accuracy: {accuracy * 100:.2f}%')

# Example of using the trained model for prediction
sample_image_path = '/kaggle/input/trials/trial/original.jpg'
sample_image = tf.keras.preprocessing.image.load_img(sample_image_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
sample_image = tf.keras.preprocessing.image.img_to_array(sample_image)
sample_image = np.expand_dims(sample_image, axis=0) / 255.0  # Rescale and expand dimensions
prediction = model.predict(sample_image)
if prediction[0] > 0.5:
    print('Prediction: Pixelated')
else:
    print('Prediction: Not Pixelated')


Found 1992 images belonging to 2 classes.
Found 498 images belonging to 2 classes.
62/62 ━━━━━━━━━━━━━━━━━━━━ 91s 1s/step - binary_accuracy: 0.5149 - loss: 0.9154 - precision_12: 0.5149 - recall_12: 0.4640 - val_binary_accuracy: 0.5021 - val_loss: 0.7030 - val_precision_12: 0.5045 - val_recall_12: 0.2333 - learning_rate: 0.0010
Model Size: 5.01 MB
78/78 ━━━━━━━━━━━━━━━━━━━━ 55s 689ms/step - binary_accuracy: 0.7320 - loss: 0.6440 - precision_12: 0.1702 - recall_12: 0.0934
Test Accuracy: 49.72%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
Prediction: Not Pixelated


In [2]:
import os
import shutil
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Define constants
IMG_HEIGHT, IMG_WIDTH = 128, 128
BATCH_SIZE = 32
EPOCHS = 10  # Increased number of epochs

# Define paths to your dataset folders
train_dir = '/kaggle/input/pixelated-image-detection-and-correction/Image_Processing/'
train_pixelated_dir = os.path.join(train_dir, 'Pixelated')
train_original_dir = os.path.join(train_dir, 'Original')

# Function to split data into train and validation directories
def split_data(original_dir, pixelated_dir, train_dir, test_dir, split_ratio):
    os.makedirs(os.path.join(train_dir, 'Original'), exist_ok=True)
    os.makedirs(os.path.join(train_dir, 'Pixelated'), exist_ok=True)
    os.makedirs(os.path.join(test_dir, 'Original'), exist_ok=True)
    os.makedirs(os.path.join(test_dir, 'Pixelated'), exist_ok=True)
    
    original_files = os.listdir(original_dir)
    pixelated_files = os.listdir(pixelated_dir)
    paired_files = [(file, file) for file in original_files if file in pixelated_files]
    np.random.shuffle(paired_files)
    split_index = int(len(paired_files) * split_ratio)
    train_files = paired_files[:split_index]
    test_files = paired_files[split_index:]
    
    for original_file, pixelated_file in train_files:
        shutil.copy(os.path.join(original_dir, original_file), os.path.join(train_dir, 'Original', original_file))
        shutil.copy(os.path.join(pixelated_dir, pixelated_file), os.path.join(train_dir, 'Pixelated', pixelated_file))
        
    for original_file, pixelated_file in test_files:
        shutil.copy(os.path.join(original_dir, original_file), os.path.join(test_dir, 'Original', original_file))
        shutil.copy(os.path.join(pixelated_dir, pixelated_file), os.path.join(test_dir, 'Pixelated', pixelated_file))

# Split the data
base_dir = '/kaggle/working/'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
split_ratio = 0.8
split_data(train_original_dir, train_pixelated_dir, train_dir, test_dir, split_ratio)

# ImageDataGenerator for augmentation and scaling
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,  # Reduced range
    width_shift_range=0.1,  # Reduced range
    height_shift_range=0.1,  # Reduced range
    shear_range=0.1,  # Reduced range
    zoom_range=0.1,  # Reduced range
    horizontal_flip=True,
    fill_mode='nearest'
)

# Generate batches of augmented data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

validation_generator = train_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

# Define the model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D((2, 2)),
    # BatchNormalization(),  # Removed for simplicity

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    # BatchNormalization(),  # Removed for simplicity

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    # BatchNormalization(),  # Removed for simplicity

    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    # BatchNormalization(),  # Removed for simplicity

    GlobalAveragePooling2D(),

    Dense(128, activation='relu'),
    Dropout(0.5),

    Dense(64, activation='relu'),
    Dropout(0.5),

    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss=BinaryCrossentropy(),
              metrics=[BinaryAccuracy(), Precision(), Recall()])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)  # Increased patience
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)
model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    callbacks=[early_stopping, reduce_lr, model_checkpoint]
)

# Save the model
model.save('/kaggle/working/pixelated_detection_model.keras')

# Calculate and print model size
model_size = os.path.getsize('/kaggle/working/pixelated_detection_model.keras') / (1024 * 1024)
print(f'Model Size: {model_size:.2f} MB')

# Evaluate the model
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

evaluation_results = model.evaluate(test_generator)
loss, accuracy, precision, recall = evaluation_results[:4]
print(f'Test Accuracy: {accuracy * 100:.2f}%')

# Example of using the trained model for prediction
sample_image_path = '/kaggle/input/pixelated-image-detection-and-correction/Image_Processing/Original/102.png'
sample_image = load_img(sample_image_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
sample_image = img_to_array(sample_image)
sample_image = np.expand_dims(sample_image, axis=0) / 255.0
prediction = model.predict(sample_image)
if prediction[0] > 0.5:
    print('Prediction: Pixelated')
else:
    print('Prediction: Not Pixelated')


Found 2390 images belonging to 2 classes.
Found 898 images belonging to 2 classes.


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 97s 1s/step - binary_accuracy: 0.5663 - loss: 0.6710 - precision_1: 0.5661 - recall_1: 0.6550 - val_binary_accuracy: 0.6942 - val_loss: 0.6055 - val_precision_1: 0.8692 - val_recall_1: 0.4588 - learning_rate: 0.0010
Epoch 4/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - binary_accuracy: 0.6875 - loss: 0.6187 - precision_1: 0.7778 - recall_1: 0.4667 - val_binary_accuracy: 1.0000 - val_loss: 0.5472 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00 - learning_rate: 0.0010
Epoch 5/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 96s 1s/step - binary_accuracy: 0.6954 - loss: 0.5919 - precision_1: 0.8810 - recall_1: 0.4806 - val_binary_accuracy: 0.6998 - val_loss: 0.5721 - val_precision_1: 0.9279 - val_recall_1: 0.4318 - learning_rate: 0.0010
Epoch 6/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - binary_accuracy: 0.6250 - loss: 0.6596 - precision_1: 0.8750 - recall_1: 0.3889 - val_binary_accuracy: 0.5000 - val_loss: 0.5301 - val_precision_1: 1.0000 - val_recall_1: 0.5000 